In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer

In [3]:
input_texts = ["def add(a, b):", "def subtract(a, b):", "def multiply(a, b):", "def divide(a, b):"]
output_texts = ["return a + b", "return a - b", "return a * b", "return a / b"]

In [4]:
model_name = "Salesforce/codet5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
prefix = "Code Prediction:"

def tokenize_data_with_masking(input_texts):
    inputs = [prefix + text for text in input_texts]
    tokenized_inputs = tokenizer(inputs, max_length=128, truncation=True, padding=True, return_tensors="pt")
    labels = tokenized_inputs.input_ids.clone()
    probability_matrix = torch.full(labels.shape, 0.15)

    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(seq.tolist(), already_has_special_tokens=True)
        for seq in labels
    ]
    special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)

    probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100 
    tokenized_inputs['labels'] = labels
    return tokenized_inputs

tokenized_data = tokenize_data_with_masking(input_texts)

In [8]:
learning_rates_to_test = [1e-5, 5e-5, 1e-4, 5e-4]

for lr in learning_rates_to_test:
    print(f"\nTesting learning rate: {lr}")

    training_args = TrainingArguments(
        output_dir=f"./dummy_results_{lr}",
        learning_rate=lr,
        per_device_train_batch_size=32,  
        per_device_eval_batch_size=32,
        num_train_epochs=5,  
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        logging_steps=1,
    )

    class DummyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings):
            self.encodings = encodings

        def __getitem__(self, idx):
            item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
            return item

        def __len__(self):
            return len(self.encodings['input_ids'])

    train_dataset = DummyDataset(tokenized_data)
    eval_dataset = DummyDataset(tokenized_data)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer
    )

    trainer.train()

    evaluation_results = trainer.evaluate(eval_dataset)
    print(f"Evaluation results for LR {lr}: {evaluation_results}")

    print(f"Final training loss for LR {lr}: {trainer.state.log_history[-1]}")

    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


Testing learning rate: 1e-05


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 20%|██        | 1/5 [00:07<00:29,  7.43s/it]

{'loss': 25.6937, 'learning_rate': 8.000000000000001e-06, 'epoch': 1.0}


                                             
 20%|██        | 1/5 [00:08<00:29,  7.43s/it]

{'eval_loss': 24.357620239257812, 'eval_runtime': 0.5406, 'eval_samples_per_second': 7.399, 'eval_steps_per_second': 1.85, 'epoch': 1.0}


...

{'eval_loss': 11.796076774597168, 'eval_runtime': 0.5714, 'eval_samples_per_second': 7.001, 'eval_steps_per_second': 1.75, 'epoch': 5.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
100%|██████████| 5/5 [01:21<00:00, 16.27s/it]


{'train_runtime': 81.3408, 'train_samples_per_second': 0.246, 'train_steps_per_second': 0.061, 'train_loss': 18.48701229095459, 'epoch': 5.0}


100%|██████████| 1/1 [00:00<00:00, 12.17it/s]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Evaluation results for LR 0.0005: {'eval_loss': 11.796076774597168, 'eval_runtime': 0.6954, 'eval_samples_per_second': 5.752, 'eval_steps_per_second': 1.438, 'epoch': 5.0}
Final training loss for LR 0.0005: {'eval_loss': 11.796076774597168, 'eval_runtime': 0.6954, 'eval_samples_per_second': 5.752, 'eval_steps_per_second': 1.438, 'epoch': 5.0, 'step': 5}
